In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os
import pickle

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

root_dir = os.getcwd() + "\\data\\reorganized"
categories = os.listdir(root_dir)
width = 224
height = 224
dim = (width, height)
data = []


In [3]:
def make_data():
    for category in categories:
        path = os.path.join(root_dir, category)
        label = categories.index(category)
        for img_name in os.listdir(path):
            image_path = os.path.join(path, img_name)
            image = cv2.imread(image_path)

            try:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, dim)
                image = np.array(image, dtype=np.float32)
                data.append([image, label])
            except Exception as e:
                pass

    pik = open('pickle/ham10000classifier.pickle', 'wb')
    pickle.dump(data, pik)
    pik.close()

In [4]:
make_data()

In [3]:
def load_data():
    pick = open('pickle/ham10000classifier.pickle', 'rb')
    data = pickle.load(pick)
    pick.close()

    feature = []
    labels = []

    for img, label in data:
        feature.append(img)
        labels.append(label)

    feature = np.array(feature, dtype=np.float32)
    labels = np.array(labels)
    feature = feature / 255.0
    return [feature, labels]

In [4]:
(feature, labels) = load_data()

In [5]:
from ModelResNet import create_model
from tensorflow.keras.optimizers import Adam

lr = 0.0001
model = create_model()
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=lr),
              metrics=['acc'])



In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [7]:
inputs = feature
targets = labels
fold_no = 1

from KFoldTrain import KFoldtrain

KFoldtrain(num_folds=3, inputs=feature, targets=labels, model=model, categories=categories)


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100


ResourceExhaustedError:  OOM when allocating tensor with shape[109,64,112,112] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/conv1_bn/FusedBatchNormV3 (defined at D:\PythonProject\HAMClassification\KFoldTrain.py:40) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_33453]

Function call stack:
train_function
